In [1]:
import os, sys, functools, itertools, collections, re, logging, json
import numpy as np
from pathlib import *
from tabulate import *
from addict import Dict

In [2]:
import matplotlib
# matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
from collections import OrderedDict

In [3]:
from IPython.display import display
from IPython.display import *


In [4]:
import PIL
import PIL.Image
from base64 import b64encode

In [5]:
NOTEBOOK_DIR = Path(os.curdir).resolve() 

In [6]:
PARENT_DIR = Path(os.curdir) / '..' 
PARENT_DIR = PARENT_DIR.resolve()
PARENT_DIR

PosixPath('/Users/elcritch/proj/code/scilab/scilab')

In [7]:
import scilab 
import scilab.tools.graphing 
import scilab.tools.jsonutils as Json
from scilab.tools.project import *
from scilab.expers.mechanical.fatigue.uts import TestInfo
from scilab.expers.mechanical.fatigue.uts import FileStructure
# from scilab.expers.mechanical.fatigue.helpers import flatten

In [13]:
WORKING_DIR_FULL = Path(os.path.expanduser("~/proj/expers/fatigue-failure|uts|expr1"))
WORKING_DIR_FULL = WORKING_DIR_FULL.resolve()
WORKING_DIR_FULL

PosixPath('/Users/elcritch/proj/expers/fatigue-failure|uts|expr1')

In [14]:
import scilab.expers.configuration as config
import scilab.expers.mechanical.fatigue.uts as exper_uts

In [15]:
pdp = WORKING_DIR_FULL / 'projdesc.json' 
fs = config.FileStructure(projdescpath=pdp,testinfo=exper_uts.TestInfo, verify=True)
# Test test images for now
test_dir = fs.tests.resolve()
testitemsd = fs.testitemsd()


In [16]:
import seaborn as sns
# sns.set_style("whitegrid")
# sns.set_style("ticks")
sns.set_style("dark")

testitems = { k.name: DataTree(info=k, folder=v, data=DataTree() ) for k,v in testitemsd.items()}
for testitem in testitems.values():
    testitem.folder = fs.testfolder(testinfo=testitem.info)
    testitem.data.details = Json.load_json_from(testitem.folder.details)


In [17]:
print(list(testitems.items())[0][1].data.details)

{'excel': {'notes': {'distance': None,
                     'failue': None,
                     'location': None,
                     'notes': None},
           'other': {'est_precond': 0.58832,
                     'est_rate': 0.14708,
                     'max_force': '-',
                     'precond_amp': 0.29416,
                     'precond_amt': 0.08,
                     'stress_uts': '#VALUE!',
                     'target_gauge': 7,
                     'target_position': None}},
 'gauge': {'base': -8.2,
           'init_position': -0.846,
           'units': 'mm',
           'value': 7.353999999999999},
 'graphs': {'modulus': {'data': {'linear_modulus': 20.60285624147392,
                                 'strain_linear': {'intercept': -3.2711740525642607,
                                                   'slope': 0.16333591122990795},
                                 'stress_linear': {'intercept': -68.04081455831898,
                                                   's

In [18]:
from scilab.tools.tables import mdBlock, mdHeader, ImageTable, MarkdownTable
from functools import partial 

In [19]:
def mapTo(func, iterable,*args,**kwargs):
    return [ (i, func(i,*args,**kwargs)) for i in iterable ]

In [20]:
tableStrs = []

def generateTable(table, headers, tablefmt='pipe'):
    return mdBlock(table.generateTable(headers=headers, tablefmt=tablefmt).format())

for name, test in testitems.items():
    
    if not test:
        continue
    
    tableStrs.append(mdHeader(2,"Test: "+test.info.name+" (%s)"%test.info.short))
    
    infoTable = MarkdownTable()

    row = ['**{}**'.format(test.info.set.upper()) ]
    row += test.info[1:] 
    infoTable.add_row( row )
    
    tableStrs.append(generateTable(infoTable, headers=test.info._fields))

    for name, data in test.data.items():
        jsonTable = MarkdownTable()
        
        for item in flatten(data,sep='.').items():
            jsonTable.add_row( item )
        
        tableStrs.append(mdHeader(4,"Detail: "+name))
        tableStrs.append(generateTable(jsonTable, headers=['Name','Value']))
    
#     # Images Table
#     table2 = ImageTable()
#     [ table2.add_row([img.name, ImgPath(img),]) 
#         for img in test.cleaned if img ]
    
#     tableStrs2.append(mdHeader(2,test.info.name))
#     tableStrs2.append(generateTable(table2, headers=['File','Image'])))

with open(str(fs.results/'UTS Test Details.v8.md'),'w') as report:
    report.write(mdHeader(1,'Test Details'))
    
    for t in tableStrs: report.write(t)
    report.write("\n\n")

In [21]:
def makeRows(test):
    row = OrderedDict()
    row['Short Name'] = '{}'.format(test.info.short)
    
#     for k,v in test.info.as_dict().items():
#         row[k] = v
    
    for name, data in test.data.items():
        for k,v in flatten(data,sep='.').items():
            row[k] = v
    
    return row

rows = [ makeRows(test) for test in testitems.values() if test ]

# find headers
headers = OrderedDict()
for row in rows:
    for k,v in row.items():
        headers[k] = None

headers = sorted(headers.keys())
display(HTML(debugger_summary("json", rows[0])))
# print(*headers,sep='\n')
# print(json.dumps(rows[0],indent=4))

Key,Value
Short Name,'902-1TR-701'
excel.notes.distance,None
excel.notes.failue,None
excel.notes.location,None
excel.notes.notes,None
excel.other.est_precond,0.58832
excel.other.est_rate,0.14708
excel.other.max_force,'-'
excel.other.precond_amp,0.29416
excel.other.precond_amt,0.08


In [ ]:
name = OrderedDict()
name["Short Name"] = "Short Name"
name["variables.preload.tracking.norm.post.preload_value"] = "Preload Value"
name["variables.preload.tracking.norm.post.preload_std"] = "Preload Std"
name["variables.uts.tracking.norm.post.disp_max_value"] = "displacement.value"
name["variables.uts.tracking.norm.post.load_max.value"] = "load.value"
name["variables.uts.tracking.norm.post.strain_max_value"] = "Strain+"
name["variables.uts.tracking.norm.post.stress_max.value"] = "Stress+"
name["variables.uts.tracking.norm.pre.load_balance"] = "load.offset"
name["measurements.area.units"] = "area.units"
name["measurements.area.value"] = "area.value"
name["measurements.depth.stdev"] = "depth.stdev"
name["measurements.depth.value"] = "depth.value"
name["measurements.width.stdev"] = "width.stdev"
name["measurements.width.value"] = "width.value"
name["gauge.base"] = "gauge.base"
name["gauge.preloaded"] = "gauge.preloaded"
name["gauge.value"] = "gauge.value"

name["info.sample"] = "info.sample"
name["info.layer"] = "info.layer"
name["info.orientation"] = "info.orientation"
name["info.wedge"] = "info.wedge"
name["info.side"] = "info.side"
name["info.set"] = "info.set"
name["info.date"] = "info.date"
name["info.run"] = "info.run"
name["info.name"] = "info.name"

for k,v in name.items():
    v = v.replace('.value','').split('.')
    name[k] = ' '.join( [ s[0].upper()+s[1:] for s in v ] )

print(*name.values(),sep='\n')
# make table

def makeTable(rows):
    table = MarkdownTable() 

    for row in rows:
        try:
            table.add_row( [ row.get(k,None) for k in name ] )
        except KeyError as err:
            missingTable.add_row( [ row.get(k,'') for k in name ] )

    return table

print([ r['info.orientation'] for r in rows ])

tableAll = makeTable(rows)

tableTr = makeTable([r for r in rows if r['info.orientation'] == 'tr'])
tableLg = makeTable([r for r in rows if r['info.orientation'] == 'lg'])

with open(str(fs.results/'UTS Test Summaries.v7.md'),'w') as report:
    report.write(mdHeader(1,'Test Summaries'))

    report.write(mdHeader(2,'Transverse'))
    tableStrs = generateTable(tableTr, headers=name.values(),tablefmt='simple')
    report.write(tableStrs)
    
    report.write(mdHeader(2,'Longitudinal'))
    tableStrs = generateTable(tableLg, headers=name.values(),tablefmt='simple')
    report.write(tableStrs)
    
    report.write("\n\n")

In [ ]:
import pandas as pd

In [ ]:
data = pd.DataFrame(tableAll.data, columns=name.values())
data.index = data["Short Name"]


In [ ]:
display(HTML(data[data['Info Orientation'] == 'lg'].to_html()))


In [ ]:
with open(str(fs.results/'UTS Test Details.v8.csv'),'w') as report:
    data.to_csv(report)
    

In [ ]:
data.to_clipboard()